In [1]:
# import sys
# sys.path.append('/media/')
# from helperToolz.helpsters import *
# from helperToolz.dicts_and_lists import *

import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import *
from skimage import measure
from skimage.measure import regionprops

In [3]:
imgs = getFilelist('/data/Aldhani/eoagritwin/fields/Fine_tune/img/', '.nc', deep=True)
labs = getFilelist('/data/Aldhani/eoagritwin/fields/Fine_tune/label/', '.tif', deep=True)

In [4]:
imgs.sort()
labs.sort()



In [5]:
lab_dict = {lab.split('/')[-1].split('.')[0]: lab for lab in labs}

img_lab_paths = [(img, lab_dict[img.split('/')[-1].split('.')[0]]) 
                 for img in imgs 
                 if img.split('/')[-1].split('.')[0] in lab_dict
]


In [12]:
xr.open_dataset(getFilelist('/data/Aldhani/eoagritwin/fields/ai4boundaries/sentinel2/images/AT', '.nc')[0])

<xarray.Dataset> Size: 8MB
Dimensions:      (time: 6, y: 256, x: 256)
Coordinates:
  * time         (time) datetime64[ns] 48B 2019-03-01 2019-04-01 ... 2019-08-01
  * x            (x) float64 2kB 4.826e+06 4.826e+06 ... 4.829e+06 4.829e+06
  * y            (y) float64 2kB 2.77e+06 2.77e+06 ... 2.767e+06 2.767e+06
Data variables:
    B2           (time, y, x) float32 2MB ...
    B3           (time, y, x) float32 2MB ...
    B4           (time, y, x) float32 2MB ...
    B8           (time, y, x) float32 2MB ...
    NDVI         (time, y, x) float32 2MB ...
    spatial_ref  int32 4B ...

In [10]:
xr.open_dataset(img_lab_paths[1][0])

<xarray.Dataset> Size: 6MB
Dimensions:      (time: 6, y: 256, x: 256)
Coordinates:
  * time         (time) int64 48B 0 1 2 3 4 5
  * x            (x) float64 2kB 4.435e+06 4.435e+06 ... 4.437e+06 4.437e+06
  * y            (y) float64 2kB 3.405e+06 3.405e+06 ... 3.402e+06 3.402e+06
Data variables:
    B2           (time, y, x) float32 2MB ...
    B3           (time, y, x) float32 2MB ...
    B4           (time, y, x) float32 2MB ...
    B8           (time, y, x) float32 2MB ...
    spatial_ref  int64 8B ...

In [11]:
paths = ['/data/Aldhani/eoagritwin/fields/segmented/Brandenburg/AI4_RGB_exclude_True_38/2023/masked_lines_touch_true_crop_touch_false_linecrop_text0.1_tbound0.7/',
         '/data/Aldhani/eoagritwin/fields/segmented/Brandenburg/AI4_RGB_exclude_True_38/2023/unmasked_text0.1_tbound0.65/']

for path, combi in zip(paths, ['0.1_0.7', '0.1_0.65']):
    files = getFilelist(path, '.tif', deep=True)
    # get tile files for prediction
    files_sub = [file for file in files if combi in file and '_pred_' in file]

    vrt = gdal.BuildVRT(path_safe(f'{path}prediction/{path.split('/')[-2]}.vrt'), files_sub)
    vrt = None

    arr = stackReader(f'{path}prediction/{path.split('/')[-2]}.vrt')
    npTOdisk(arr, f'{path}prediction/{path.split('/')[-2]}.vrt', f'{path}prediction/{path.split('/')[-2]}.tif', bands=1, noData=0)




In [5]:
path.split('/')[-2]

'masked_lines_touch_true_crop_touch_false_linecrop_text0.1_tbound0.7'

In [ ]:
# make a physical copy of these files
outpath = path_safe(f'{path}quick_n_dirty/')
if len(getFilelist(outpath, '.tif')) != 0:
    for file in getFilelist(outpath, '.tif'):
        os.remove(file)
for file in files_sub:
    shutil.copy2(file, outpath + file.split('intermediates/')[-1])

files_sub = getFilelist(outpath, '.tif')

In [ ]:
files_sub

In [ ]:
# loop over files to get row and col cuts
rows, cols = [], []
for file in files_sub:
    rows.append(int(file.split('_instance_pred_')[-1].split('_')[0]))
    cols.append(int(file.split('_instance_pred_')[-1].split('_')[-1].split('.')[0]))
rows.sort()
cols.sort()
rows = list(set(rows))
cols = list(set(cols))
rows.sort()
cols.sort()

In [ ]:

# transform unique-pairs array to dict
def unique_dict(unique_pairs_array):
    valid_dict = {}

    for key, value in unique_pairs_array:
        if key in valid_dict:
            valid_dict[key].append(value)
        else:
            valid_dict[key] = [value]

    return valid_dict

def make2000000(x):
    s = str(x)
    if len(s) == 7:
        return int('2' + s[1:] )
    else:
        return int('2' + s[2:] )
    

In [ ]:
##### iterate over possible row/col connections and search for fields in neighbouring tiles
# start at top-left-corner
for row in rows:
    for col in cols:

        # row = rows[0]
        # col = cols[5]
        tile1 = [sub for sub in files_sub if f'_{row}_{col}.tif' in sub]
        if len(tile1) == 0:
            continue
        else: 
            ### check for neighbouring tiles and store result in list
            #print(f'_{row}_{col}.tif')
            # upper  (row -1)
            if rows.index(row)!=0:
                upper = [sub for sub in files_sub if f'_{rows[rows.index(row)-1]}_{col}.tif' in sub]
            else:
                upper = []

            # upper right (row -1 and col +1)
            if rows.index(row)!=0 and cols.index(col) < len(cols) - 1:
                upper_right = [sub for sub in files_sub if f'_{rows[rows.index(row)-1]}_{cols[cols.index(col)+1]}.tif' in sub]
            else:
                upper_right = []

            # right (col +1)
            if cols.index(col) < len(cols) - 1:
                right = [sub for sub in files_sub if f'_{row}_{cols[cols.index(col)+1]}.tif' in sub]
            else:
                right = []

            # lower right (row +1 and col +1)
            if rows.index(row) < len(rows) - 1 and cols.index(col) < len(cols) - 1:
                lower_right = [sub for sub in files_sub if f'_{rows[rows.index(row)+1]}_{cols[cols.index(col)+1]}.tif' in sub]
            else:
                lower_right = []

            # bottom (row +1)
            if rows.index(row) < len(rows) - 1:
                bottom = [sub for sub in files_sub if f'_{rows[rows.index(row)+1]}_{col}.tif' in sub]
            else:
                bottom = []

            if any(len(lst) > 0 for lst in [upper, upper_right, right, lower_right, bottom]):
                # load starting tile
                ds = gdal.Open(tile1[0])
                dat = ds.GetRasterBand(1).ReadAsArray()
                
                ### check the other tiles
                # upper
                if len(upper) == 1:
                    ds = gdal.Open(upper[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[0,:], neighbour[-1,:]), axis=1), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all(axis=1)]
                    valid_dict = unique_dict(valid_pairs)
                    for v, p_list in valid_dict.items():
                        dat[dat == v] = 2000000 + v
                        for p in p_list:
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, upper[0], upper[0], noData=0)

                # upper right
                if len(upper_right) == 1:
                    ds = gdal.Open(upper_right[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[0,-1], neighbour[-1,0])), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all()]
                    if len(valid_pairs) > 1:
                        for v, p in valid_pairs:
                            dat[dat == v] = 2000000 + v
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, upper_right[0], upper_right[0], noData=0)

                # right
                if len(right) == 1:
                    ds = gdal.Open(right[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[:,-1], neighbour[:,0]), axis=1), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all(axis=1)]
                    valid_dict = unique_dict(valid_pairs)
                    for v, p_list in valid_dict.items():
                        dat[dat == v] = 2000000 + v
                        for p in p_list:
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, right[0], right[0], noData=0)

                # lower right
                if len(lower_right) == 1:
                    ds = gdal.Open(lower_right[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()
                    
                    unique_pairs = np.unique(np.stack((dat[-1,-1], neighbour[0,0])), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all()]
                    if len(valid_pairs) > 1:
                        for v, p in valid_pairs:
                            dat[dat == v] = 2000000 + v
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, lower_right[0], lower_right[0], noData=0)

                # bottom
                if len(bottom) == 1:
                    ds = gdal.Open(bottom[0])
                    neighbour = ds.GetRasterBand(1).ReadAsArray()

                    unique_pairs = np.unique(np.stack((dat[-1,:], neighbour[0,:]), axis=1), axis=0)
                    valid_pairs = unique_pairs[(unique_pairs != 0).all(axis=1)]
                    valid_dict = unique_dict(valid_pairs)
                    for v, p_list in valid_dict.items():
                        dat[dat == v] = 2000000 + v
                        for p in p_list:
                            neighbour[neighbour == p] = 2000000 + v
                    # export manipulated tile (will overwrite)
                    makeTif_np_to_matching_tif(neighbour, bottom[0], bottom[0], noData=0)

                makeTif_np_to_matching_tif(dat, tile1[0], tile1[0], noData=0)
            else:
                continue

 


In [ ]:
for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            # Delete files or symlinks
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            # Recursively delete directories
            shutil.rmtree(file_path)
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")

In [ ]:
year = 2023
model_name = 'AI4_RGB_exclude_False_47'
ncores = 56

pred_list_sorted = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/{model_name}/{year}/vrt/', '.vrt') if 'unmasked' in file]
ref_list_sorted = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/IACS/4_Crop_mask/{year}/', '.tif') if all(crit in file for crit in ['linecrop', 'prediction_extent', 'lines_touch_true_crop_touch_true'])]
# overlords_jobs = []


print(ref_list_sorted)


for prediction, reference in zip(pred_list_sorted, ref_list_sorted):
    if 'unmasked' in prediction:
        result_dir = path_safe(f'/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/{model_name}/{year}/' + prediction.split('/')[-1].split('.')[0] + reference.split('/')[-1].split('.')[0])
    else:
        result_dir = path_safe(f'/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/{model_name}/{year}/' + reference.split('/')[-1].split('.')[0])

In [ ]:
year = 2023

pred_list = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/{year}/vrt/', '.vrt') if 'unmasked' not in file]
ref_list = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/fields/IACS/4_Crop_mask/{year}/', '.tif') if 'prediction_extent' in file]

pred_list_sorted, ref_list_sorted = [], []
for pred in pred_list:
    for ref in ref_list:
        if '_'.join(pred.split('/')[-1].split('_')[:-2]) == ref.split('cropMask_')[-1].split('_prediction_extent')[0]:
            pred_list_sorted.append(pred)
            ref_list_sorted.append(ref)
        else:
            pass

In [ ]:
prediction = pred_list_sorted[0]
reference = ref_list_sorted[0]
reference.split('/')[-1].split('.')[0]

In [ ]:
for prediction, reference in zip(pred_list_sorted, ref_list_sorted):

    # set the number by which rows and cols will be divided --> determines the number of tiles // also set border limit (dont sample fields too close to tile borders) and sample size
    slicer = 10
    border_limit = 5
    sample_size  = 10000
    # set the number of cores for parallel processing and set seed
    ncores = 55
    np.random.seed(42)
    make_tifs_from_intermediate_step = True

    ######### prepare job-list

    # create lists that will be passed on to the joblist
    # tile_list = []
    extent_true_list = []
    extent_pred_list = []
    boundary_pred_list = []
    result_dir_list = []
    row_col_start = []


    # tile predictions in prds --> total extent encompasses 90 Force Tiles (+ a few rows and cols that will be neglected as they are outside of study area)
    pred_ds = gdal.Open(prediction)
    rows, cols = pred_ds.RasterYSize, pred_ds.RasterXSize

    row_start = [i for i in range(0, rows, math.floor(rows/slicer))]
    row_end = [i for i in range (math.floor(rows/slicer), rows, math.floor(rows/slicer))]
    row_start = row_start[:len(row_end)] 

    col_start = [i for i in range(0, cols, math.floor(cols/slicer))]
    col_end = [i for i in range (math.floor(cols/slicer), cols, math.floor(cols/slicer))]
    col_start = col_start[:len(col_end)] 

        # load IACS reference mask and label it 
    ref_ds = gdal.Open(reference)
    extent_true = ref_ds.GetRasterBand(1).ReadAsArray()
    binary_true = extent_true > 0
    instances_true = measure.label(binary_true, background=0, connectivity=1)

    # sample fields
    # build a mask to exclude fields that are in border_limit to tile borders
    power_mask = np.zeros(instances_true.shape)
    for i in range(len(row_end)):
        for j in range(len(col_end)):
                power_mask[row_start[i]:row_start[i] + border_limit, :] = 1
                power_mask[:, col_start[j]:col_start[j] + border_limit] = 1
                power_mask[row_end[-1] - border_limit:power_mask.shape[0], :] = 1
                power_mask[:, col_end[-1] - border_limit:power_mask.shape[1]] = 1

    # get IDs from labelled reference
    IDs_to_skip = np.unique(instances_true[power_mask==1])

    # get distribution of field sizes after segmentation
    unique_IDs, counts = np.unique(instances_true, return_counts=True)

    # exlcude fields that are too close to tile borders
    mask = ~np.isin(unique_IDs, IDs_to_skip)
    unique_IDs = unique_IDs[mask]
    counts = counts[mask]

    # exlude 0 (background) and super-small fields (~2 pixel) from sample
    pixelthresh = 0

    while True:
        mask = (unique_IDs != 0)  & (counts > pixelthresh)
        unique_IDs = unique_IDs[mask]
        counts = counts[mask]

        # get deciles and draw equally from them
        deciles = [perc for perc in range(10,100,10)]
        deciles_values = np.percentile(counts, deciles)
        decs = [0] + deciles_values.tolist() + [np.max(counts)]

        if len(decs) == len(set(decs)):
            df = pd.DataFrame({'decile_value': decs,
                   'excluded_pixel':pixelthresh})
            df.to_csv("decs_output.csv", index=False)
            break
        else:
            pixelthresh += 1

In [ ]:
bin_ids = []
for ind in range(len(decs) -1):
    # get the unique_IDS of those fields, whose count (size) is within bin
    bin_ids.append(np.random.choice(unique_IDs[(counts > decs[ind]) & (counts <= decs[ind + 1])], int(sample_size/10), replace=False))



In [ ]:
instances_true = np.where(np.isin(instances_true, np.concatenate(bin_ids)),
                        instances_true,
                        0)

In [ ]:
np.unique(instances_true)

In [ ]:
c = 0
t_exts = [i/100 for i in range(10,95,5)] 
t_bounds = [i/100 for i in range(10,95,5)]
for t_ext in t_exts:
    for t_bound in t_bounds:
        c += 1
print(c)

In [ ]:

bin_ids = []
for ind in range(len(decs) -1):
    # get the unique_IDS of those fields, whose count (size) is within bin
    bin_ids.append(np.random.choice(unique_IDs[(counts > decs[ind]) & (counts <= decs[ind + 1])], int(sample_size/10), replace=False))

instances_true = np.where(np.isin(instances_true, np.concatenate(bin_ids)),
                        instances_true,
                        0)


In [ ]:
pathi = '/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/2023/test1/unmasked_chips_256_20GSA-DE_BRB-2023_cropMask_lines_touch_true_crop_touch_true_linecrop_prediction_extent/intermediates/'


instances_pred = stackReader(checkPath(f'{pathi}0.4_0.2_instance_pred_10760_17982.tif'))
instances_true = stackReader(checkPath(f'{pathi}0.4_0.2_instance_true_10760_17982.tif'))


In [ ]:
row_col_start = '10760_17982'

# --- 1) get predicted instance segmentation and relabel to 1..N (background 0) ---
instances_pred_raw = instances_pred

# Robust labeling: treat zeros as background; produce labels 0..P
binary_pred = instances_pred_raw != 0
instances_pred = measure.label(binary_pred, background=0)  # labels start at 1 for objects

# Flatten arrays once for a single-pass overlap computation
flat_true = instances_true.ravel()
flat_pred = instances_pred.ravel()

max_true_label = int(flat_true.max()) if flat_true.size else 0
max_pred_label = int(flat_pred.max()) if flat_pred.size else 0

# Build intersection matrix: rows=true-label, cols=pred-label
# intersection[i, j] = number of pixels where true==i and pred==j
# allocate (max_true_label+1, max_pred_label+1) to include the zero label row/col
intersection = np.zeros((max_true_label + 1, max_pred_label + 1), dtype=np.int32)

# Count only where both are non-zero (optional: we will still fill zeros row/col but it's faster to limit)
valid_mask = (flat_true > 0) & (flat_pred > 0)
if np.any(valid_mask):
    np.add.at(intersection, (flat_true[valid_mask], flat_pred[valid_mask]), 1)

# Compute per-label areas using bincount (includes zero index)
area_true = np.bincount(flat_true, minlength=(max_true_label + 1)).astype(np.int32)
area_pred = np.bincount(flat_pred, minlength=(max_pred_label + 1)).astype(np.int32)


In [ ]:
area_true

In [ ]:

# Avoid division by zero: union = A_true + A_pred - intersection
# Vectorized IoU table (shape: (max_true_label+1, max_pred_label+1))
# We'll compute IoU only for labels >0 later; but it's fine to compute whole table.
union = (area_true[:, None] + area_pred[None, :] - intersection).astype(np.int32)
# protect against zero union by setting union to 1 where zero (IoU will be 0 because intersection is also 0)
zero_union_mask = union == 0
union_safe = union.copy()
union_safe[zero_union_mask] = 1
IoU_table = intersection / union_safe.astype(np.float32)
IoU_table[zero_union_mask] = 0.0  # ensure 0 where union was actually zero

# --- 3) prepare outputs in the same order the original iterated (np.unique order) ---
field_values = np.unique(instances_true)
# allocate lists
best_IoUs = []
centroid_rows = []
centroid_cols = []
centroid_IDs = []
centroid_IoUS = []
field_IDs = []
field_sizes = []
intersect_IDs = []
pred_field_overlap = []

# use regionprops for centroid & area extraction (map label -> region)
props_true = regionprops(instances_true)  # returns list of RegionProperties for labels > 0
# create mapping label -> RegionProperties for quick access
props_map = {p.label: p for p in props_true}

# we'll also need to get predicted label areas quickly (area_pred array already computed)
# and to access instances_pred by centroid coordinates

for fv in field_values:
    if fv == 0:
        continue

    field_IDs.append(int(fv))
    # area of the true field (from bincount)
    f_area = int(area_true[int(fv)])
    field_sizes.append(f_area)

    # centroid from regionprops (fallback if missing)
    if fv in props_map:
        centroid = props_map[fv].centroid  # (row, col) floats
        r = int(round(centroid[0]))
        c = int(round(centroid[1]))
    else:
        # fallback: compute centroid from coordinates (rare)
        coords = np.column_stack(np.where(instances_true == fv))
        if coords.size == 0:
            r, c = 0, 0
        else:
            mean_coords = np.mean(coords, axis=0)
            r, c = int(round(mean_coords[0])), int(round(mean_coords[1]))

    centroid_rows.append(int(r))
    centroid_cols.append(int(c))

    # find best predicted partner by IoU for this true label
    # Only consider pred labels >=1 .. max_pred_label
    if int(fv) <= max_true_label and max_pred_label >= 1:
        row_iou = IoU_table[int(fv), 1:(max_pred_label + 1)]  # exclude pred label 0
        if row_iou.size == 0 or np.all(row_iou == 0):
            best_j_rel = -1
            best_iou_val = 0.0
            best_pred_label = 0
        else:
            best_j_idx = np.argmax(row_iou)  # index in row_iou
            best_iou_val = float(row_iou[best_j_idx])
            best_pred_label = int(1 + best_j_idx)  # convert back to actual pred label
    else:
        best_pred_label = 0
        best_iou_val = 0.0

    best_IoUs.append(best_iou_val)
    intersect_IDs.append(int(best_pred_label))

    # compute predicted-field overlap fraction: intersection / pred_field_area
    if best_pred_label > 0:
        inter_area = int(intersection[int(fv), int(best_pred_label)])
        pred_area = int(area_pred[int(best_pred_label)]) if int(best_pred_label) <= max_pred_label else 0
        overlap_frac = (inter_area / pred_area) if pred_area > 0 else 0.0
    else:
        overlap_frac = 0.0
    pred_field_overlap.append(overlap_frac)

    # centroid-based predicted id and IoU
    # ensure centroid coordinates are inside bounds
    hr, hc = instances_pred.shape
    if 0 <= r < hr and 0 <= c < hc:
        pred_at_centroid = int(instances_pred[r, c])
    else:
        pred_at_centroid = 0

    centroid_IDs.append(pred_at_centroid)
    # centroid IoU: IoU_table[true_label, pred_at_centroid] if pred_at_centroid>0 else 0
    if pred_at_centroid > 0 and int(fv) <= max_true_label and pred_at_centroid <= max_pred_label:
        centroid_iou_val = float(IoU_table[int(fv), pred_at_centroid])
    else:
        centroid_iou_val = 0.0
    centroid_IoUS.append(centroid_iou_val)